In [1]:
from zerodha_profolio_tool import login,generate_session,get_current_holdings


## Local callback server
This notebook starts a small HTTP server on `127.0.0.1:<port>` to receive the redirect after Zerodha login.
1. Start the server cell (choose a free `port`, e.g. `8080`).
2. Call `login(redirect_uri='http://127.0.0.1:8080')` from the `zerodha_profolio_tool` module (it will open the login URL).
3. After completing login in the browser, Zerodha will redirect to the local server and the notebook will capture `request_token`.


In [ ]:
# Implement a tiny callback HTTP server to capture the `request_token`
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading
from urllib.parse import urlparse, parse_qs
import time

_token_data = {}

class TokenHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        parsed = urlparse(self.path)
        qs = parse_qs(parsed.query)
        token = qs.get('request_token') or qs.get('requestToken')
        if token:
            _token_data['request_token'] = token[0]
            message = '<html><body><h2>Login successful. You can close this tab.</h2></body></html>'
        else:
            message = '<html><body><h2>No request_token found in URL.</h2></body></html>'
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()
        self.wfile.write(message.encode('utf-8'))

def start_server(port: int = 8080):
    server = HTTPServer(('127.0.0.1', port), TokenHandler)
    thread = threading.Thread(target=server.serve_forever, daemon=True)
    thread.start()
    return server

def get_request_token(timeout: int = 300):
    start = time.time()
    while time.time() - start < timeout:
        if 'request_token' in _token_data:
            return _token_data.pop('request_token')
        time.sleep(0.5)
    raise TimeoutError('No request_token received within timeout')

def stop_server(server: HTTPServer):
    try:
        server.shutdown()
        server.server_close()
    except Exception as e:
        print('Error stopping server:', e)


In [ ]:
# Example usage: start the server, call login(), then wait for token.
# 1) Start server on a free port (use same port in redirect URI).
server = start_server(port=8080)
print('Server started at http://127.0.0.1:8080')

# 2) Trigger Zerodha login (this opens the browser). Make sure your redirect URI is set to http://127.0.0.1:8080
# from zerodha_profolio_tool import login
login_url, kite = login(open_in_browser=True)

kite,access= generate_session(kite, request_token=_token_data.get('request_token'))
# Uncomment and run the above two lines after you have `KITE_API_KEY` and `KITE_API_SECRET` set in environment or .env.

# 3) Wait for request token (this will block until token is received or timeout)
# request_token = get_request_token(timeout=300)
# print('Received request_token:', request_token)

# 4) Exchange request_token for access token (example):
# data = kite.generate_session(request_token, os.getenv('KITE_API_SECRET'))
# access_token = data['access_token']
# kite.set_access_token(access_token)
# print('Access token:', access_token)


In [ ]:
get_current_holdings(kite)